In [1]:
import tensorflow; tensorflow.__version__

'1.2.0'

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
NCLASSES = 10
y_train = keras.utils.to_categorical(y_train, NCLASSES)
y_test = keras.utils.to_categorical(y_test, NCLASSES)

In [6]:
## Using more advanced data augmentation prevents the model from generalizing.
## val_acc gets stuck around 10%, even though acc reaches 40%
# train_datagen = ImageDataGenerator(
# featurewise_center=True,
# samplewise_center=True,
# featurewise_std_normalization=True,
# rotation_range=10,
# zoom_range = 0.9,
# horizontal_flip=True
# )

train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train, y_train)
train_generator.samples = x_train.shape[0]

In [7]:
val_datagen=ImageDataGenerator()
val_generator = val_datagen.flow(x_test, y_test)
val_generator.samples = x_test.shape[0]

In [8]:
def makemodel():
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    model = Sequential()
    # input: 32, 32 images with 3 channels -> (32,32, 3) tensors.
    # this applies 32 convolution filters of size 3x3 each.
    model.add(Conv2D(256, (6, 6), activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(128, (6, 6), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NCLASSES, activation='softmax'))
    return model

In [9]:
from keras.models import load_model
model = load_model('cifar10-cb.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 27, 27, 256)       27904     
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 22, 22, 128)       1179776   
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 11, 11, 128)       0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 9, 9, 128)         147584    
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 3, 3, 128)         0         
__________

In [69]:
# from keras.applications.resnet50 import ResNet50
# model = ResNet50(weights=None, classes=10)
model = makemodel()
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.rmsprop(lr=0.0001, decay=1e-6), metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_57 (Conv2D)           (None, 27, 27, 256)       27904     
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 22, 22, 128)       1179776   
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 11, 11, 128)       0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 9, 9, 128)         147584    
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 3, 3, 128)         0         
__________

In [ ]:
saver_cb = keras.callbacks.ModelCheckpoint(
    'cifar10-cb.h5',
    monitor='val_acc',
    verbose=1,
    save_best_only=True
)
model.fit_generator(
    train_generator,
    steps_per_epoch=int(train_generator.samples/train_generator.batch_size),
    epochs=200,
    validation_data=val_generator,
    validation_steps=int(val_generator.samples/val_generator.batch_size),
    initial_epoch=20,
    callbacks=[saver_cb]
)

Epoch 21/200
 329/1562 [=====>........................] - ETA: 61s - loss: 1.8006 - acc: 0.4046

Goal: reach about 0.70 val_acc.
# Observations
I noticed some divergence between acc and val_acc. For example, in Epoch 23, acc was 0.40 but val_acc was .50. If there is less correlation, then training for more epochs won't necessarily help!
Accuracy collaposed after Epoch 25 back to 0.1 (guessing). Need to add in variable LR.